In [1]:
import json
import os
import cv2
import numpy as np
import math
from glob import glob
from PIL import Image, ExifTags

# dataset path
- 데이터 구조
  - <데이터 셋 이름> 폴더
    - exif_images <변경 전>
    - images <변경 후>
    - UFO
      - train.json

# Config

In [16]:
data_dir = '/opt/ml/input/data/hiertext'  ## datasets path

save_path = '/opt/ml/input/data/hiertext'

if not os.path.exists(os.path.join(save_path, 'images')):
    os.makedirs(os.path.join(save_path, 'images'))

with open(os.path.join(data_dir, 'ufo/{}.json'.format('train')), 'r') as f:
    anno = json.load(f)
    
image_fnames = sorted(anno['images'].keys())

print(len(image_fnames))

8281


# tag 정보 불러와서 이미지 회전시키고 저장

In [19]:
for file in image_fnames:
    image_path = os.path.join(data_dir, 'exif_images', file)
    image = Image.open(image_path)
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation]=='Orientation':
                break
        exif=dict(image._getexif().items())
        if exif[orientation] == 3:
            image=image.rotate(180, expand=True)
        elif exif[orientation] == 6:
            image=image.rotate(270, expand=True)
        elif exif[orientation] == 8:
            image=image.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # cases: image don't have getexif
        pass
    extension = file.split('.')[-1]
    if extension == 'jpg' or extension == 'JPG':
        extension = 'JPEG'
    image.save(os.path.join(save_path,'images',file) , extension)